# 
<a id="data-prep"></a>
<div style="background-color: #000D5B; color: white; text-align: center; padding: 6px 0 22px 0">
    <h3 style="background-color: #000D5B; color: white; text-align: left">RMIT School of Computer Science and Technology</h3>
    <br/>
    <h1>COSC3007: Deep Learning</h1>
    <h2>Assignment 2: Stance Twitter Sentiment Analysis and Detection </h2>
    <h2> [2] MODELLING AND MODEL EVALUATIONS </h2>

</div>

# Import Libs

In [40]:
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import numpy as np

# [1] Load Data

In [41]:
train_df = pd.read_csv("./StanceDataset/processed_train.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])
test_df = pd.read_csv("./StanceDataset/processed_test.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])

# [2] Prepare label and fit data

## Split test and val

In [42]:
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

## Tweets

In [43]:
MAX_LENGTH = 123

# Train
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['processed_tweet'])
train_sequences = tokenizer.texts_to_sequences(train_df['processed_tweet'])

# Test
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(train_df['processed_tweet'])
test_sequences = tokenizer2.texts_to_sequences(test_df['processed_tweet'])

# Val
tokenizer3 = Tokenizer()
tokenizer3.fit_on_texts(train_df['processed_tweet'])
val_sequences = tokenizer3.texts_to_sequences(val_df['processed_tweet'])

train_pad = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post')
test_pad = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post')
val_pad = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding='post')

word_index = tokenizer.word_index
print("unique tokens - "+str(len(word_index)))
vocab_size = len(tokenizer.word_index) + 1
print('vocab_size - '+str(vocab_size))

unique tokens - 7518
vocab_size - 7519


## Stances

In [44]:
def categorized_label(df, label_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df[label_name])
    categorical_labels = to_categorical(encoded_labels)
    return categorical_labels

In [45]:
train_labels = categorized_label(train_df, "Stance")
test_labels = categorized_label(test_df, "Stance")
val_labels = categorized_label(val_df, "Stance")

# [3] Create Model

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras import backend as K
from tensorflow.keras.optimizers.legacy import Adam


In [47]:
# Define the F1 score metric
def f1_score(y_true, y_pred):
    # Calculate Precision and Recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    # Calculate F1 score
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Set up call backs

In [48]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Define the EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-5,
    verbose=1,
)

# Define the ModelCheckpoint callback
checkpoint_path = "best_model2.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,    # Only save a model if 'val_loss' has improved
    monitor='val_loss',     # Monitor 'val_loss' during training
    mode='min',             # The model is saved when 'val_loss' is minimized
    verbose=1)

# Combine all callbacks in a list
callbacks = [
    early_stopping,
    lr_scheduler,
    # model_checkpoint_callback
]

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

In [49]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=16, input_length=MAX_LENGTH))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # 3 classes: FAVOUR, AGAINST, NEITHER

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', f1_score])

In [50]:
history = model.fit(train_pad, train_labels, epochs=10, validation_data=(val_pad, val_labels), batch_size=32, callbacks=callbacks)

Epoch 1/10


2023-12-20 11:32:31.931205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


23/92 [======>.......................] - ETA: 41:14 - loss: 1.0792 - accuracy: 0.4592 - f1_score: 0.0124